<h2><center>Week 5 - Assignment</center></h2>
<h3><center>Programming for Data Science 2024</center></h3>

Exercises for the topics covered in the fifth lecture.

The exercise will be marked as passed if you get **at least 10/15** points.

Exercises must be handed in via **ILIAS** (Homework assignments). Deliver your submission as a compressed file (zip) containing one .py or .ipynb file with all exercises. The name of both the .zip and the .py/.ipynb file **must** be *SurnameName* of the two members of the group. Example: Riccardo Cusinato + Athina Tzovara = *CusinatoRiccardo_TzovaraAthina.zip* .

It's important to use comments to explain your code and show that you're able to take ownership of the exercises and discuss them.

You are not expected to collaborate outside of the group on exercises and submitting other groups’ code as your own will result in 0 points.

For questions contact: *riccardo.cusinato@unibe.ch* with the subject: *Programming for Data Science 2024*.

**Deadline: 14:00, March 28, 2024.**

<h3 style="text-align:left;">Exercise 1 - Fitbit dataset<span style="float: right">3 points</span></h3>

We will work with three datasets - 'activity.csv', 'calories.csv', and 'last_participant.csv', which contains activity tracker data from https://www.kaggle.com/datasets/arashnic/fitbit

If you are unable to do this exercise, you can load the dataset 'combined_solution.csv' for the next exercise. 

1. **Data preparation** (*1 point*)

    - Load the two datasets 'activity.csv' and 'calories.csv'.
    - Use pd.to_datetime to standardize the ActivityDate columns (https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [162]:
import pandas as pd

def standardize_activity_date(df: pd.DataFrame):
    df['ActivityDate'] = pd.to_datetime(df['ActivityDate']) # no format specified, since multiple formats exist in the data and the inferred format seems to be correct everywhere

activity_df: pd.DataFrame = pd.read_csv('./data/activity.csv')
calories_df: pd.DataFrame = pd.read_csv('./data/calories.csv')
standardize_activity_date(activity_df)
standardize_activity_date(calories_df)
calories_df


C:\Users\lukik\AppData\Local\Temp\ipykernel_18400\704096592.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ActivityDate'] = pd.to_datetime(df['ActivityDate']) # no format specified, since multiple formats exist in the data and the inferred format seems to be correct everywhere
C:\Users\lukik\AppData\Local\Temp\ipykernel_18400\704096592.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ActivityDate'] = pd.to_datetime(df['ActivityDate']) # no format specified, since multiple formats exist in the data and the inferred format seems to be correct everywhere


,ID,ActivityDate,Calories
0,1624580081,2016-04-12,1432.0
1,1624580081,2016-04-13,1411.0
2,1624580081,2016-04-14,1572.0
3,1624580081,2016-04-15,1344.0
4,1624580081,2016-04-16,NaN
...,...,...,...
904,8792009665,2016-05-06,1688.0
905,8792009665,2016-05-07,1688.0
906,8792009665,2016-05-08,1688.0
907,8792009665,2016-05-09,1688.0


2. **Merging** (*1 point*)

    - Consider what information is shared between the two datasets and merge them. Keep in mind that the order of rows is not the same in both datasets!
    - Print out the mean "TotalSteps" of the merged DataFrame at this point.

In [163]:
combined_df: pd.DataFrame = pd.merge(activity_df, calories_df, on=['ID', 'ActivityDate']) # id and activityDate is in both datasets and together are a unique key for each row, since both id and on their own are not unique, there are multiple entries with the same date.

print("Mean TotalSteps: "+str(combined_df['TotalSteps'].mean()))


Mean TotalSteps: 7786.438763376932


3. **Concatenation** (*1 point*)

    - The data of one additional participant exists in 'last_participant.csv'. Load this dataset and concatenate it with the merged dataset generated above
    - Print out the mean "TotalSteps" again

In [164]:
last_participant_df: pd.DataFrame = pd.read_csv('./data/last_participant.csv')
combined_df = pd.concat([combined_df, last_participant_df])

print("Mean TotalSteps last participant included: "+str(combined_df['TotalSteps'].mean()))

Mean TotalSteps last participant included: 7879.460599334073


<h3 style="text-align:left;">Exercise 2 - Working with missing data<span style="float: right">5 points</span></h3>

In our dataset, some values are missing from the 'TotalSteps' and 'Calories' columns.

We can try to approximate these missing values with the data we got. 

You can load the dataset 'combined_solution.csv' if you were unable to complete the previous exercise.

1. **Filling in missing values** (*3 points*)

    - Calculate the mean steps per calory burnt and mean calories burnt per step, by averaging across all observations in the dataset and then computing the ratio. Print out both values.
    - Fill in the null values in the columns 'Calories' and 'TotalSteps' where possible. To fill the values you have to use the factors *"TotalSteps/Calories"* and *"Calories/TotalSteps"* calculated in the previous point, using one of the two information to fill the other.
    - Print out the mean of the columns 'TotalSteps' and 'Calories' before and after filling the missing values.

In [165]:
#calculating the mean of the steps and calories and their ratios
steps_mean = combined_df['TotalSteps'].mean()
calories_mean = combined_df['Calories'].mean()
steps_per_calory = steps_mean / calories_mean
calories_per_step = calories_mean / steps_mean
print("steps/calory: "+str(steps_per_calory))
print("calories/step: "+str(calories_per_step))

# filling the gaps in the data where possible, to calculate missing steps, one can take the calories burnt of the same row and multiply it by the average  steps/calory, since calories*(steps/calories) = steps. The same but the other way around for the missing calories.
combined_df['TotalSteps'] = combined_df['TotalSteps'].fillna(value=combined_df['Calories'] * steps_per_calory) 
combined_df['Calories'] = combined_df['Calories'].fillna(value=combined_df['TotalSteps'] * calories_per_step)

#printing the old and the new means
print('Steps mean before filling: '+str(steps_mean))
print('Calories mean before filling: '+str(calories_mean))
print('Steps mean after filling: '+str(combined_df['TotalSteps'].mean()))
print('Calories mean after filling: '+str(combined_df['Calories'].mean()))


steps/calory: 3.4100446727062272
calories/step: 0.29325129022617624
Steps mean before filling: 7879.460599334073
Calories mean before filling: 2310.661987041037
Steps mean after filling: 7785.911198419612
Calories mean after filling: 2302.939349901768


2. **Dropping missing values** (*2 points*)

    - Print how many null values there are in the 'Calories' and 'TotalSteps' columns, respectively.
    - Drop the rows where **both** 'Calories' and 'TotalSteps' are missing.
    - Print number of rows in the final dataset.

In [166]:
###
# YOUR CODE HERE
###

<h3 style="text-align:left;">Exercise 3 - Multi-index<span style="float: right">7 points</span></h3>

In this exercise you will create and manipulate a multi-index dataframe. First, let's create the dataframe for the exercise:

In [167]:
import pandas as pd

df = pd.DataFrame(
    {
        "idx": [0, 1, 2],
        "A_X": [1.1, 1.1, 1.1],
        "A_Y": [1.2, 1.2, 1.2],
        "B_X": [1.11, 1.11, 1.11],
        "B_Y": [1.22, 1.22, 1.22],
    }
)
print(df)

   idx  A_X  A_Y   B_X   B_Y
0    0  1.1  1.2  1.11  1.22
1    1  1.1  1.2  1.11  1.22
2    2  1.1  1.2  1.11  1.22


1. Set the column *idx* as the index of the dataframe. (*1 point*)

In [168]:

df_reindexed = df.set_index('idx') # set explicit index 
print(df_reindexed)

     A_X  A_Y   B_X   B_Y
idx                      
0    1.1  1.2  1.11  1.22
1    1.1  1.2  1.11  1.22
2    1.1  1.2  1.11  1.22


2. Create a multi-column stucture. (*3 points*)
    - Set the columns *A, B* on the first level and *X, Y* on the second level, taken from the combinations in the original dataframe. 
    - Set the names of the two new levels as "L1" and "L2", respectively. 
    - Print the resulting dataframe.

In [169]:
# Define Columns for multi-column structure

columns = pd.MultiIndex.from_product([['A','B'], ['X', 'Y']], names = ['L1', 'L2'])

#set new columns to dr_reindexed
df_reindexed.columns = columns

# print result
print(df_reindexed)

L1     A          B      
L2     X    Y     X     Y
idx                      
0    1.1  1.2  1.11  1.22
1    1.1  1.2  1.11  1.22
2    1.1  1.2  1.11  1.22


3. From the previous dataframe, re-create a dataframe with a single column level. (*3 points*)
    - Create a new column from the first level (L1) of the multi-column. At this point your columns should be ['L1', 'X', 'Y'], with name 'L2'. **NB** The DataFrame method *reset_index* is useful for this part.
    - Rename the newly-created column as "letter" and the name of the column level as "L". Use the appropiate pandas methods for this.
    - Print the resulting dataframe.

In [170]:
# stack DataFrame, such that L1 is a separate column with A and B as inputs
df_stacked = df_reindexed.stack(level = 0)

# reindex  df_stacked, such that the columns are L1, X, Y and the name is L2

df = df_stacked.reset_index(level = 1)
# rename L1 as letter
df = df.rename(columns={"L1":"letter"})

# set L2 to L
df = df.rename_axis('L', axis= 1 )

print(df)














L   letter     X     Y
idx                   
0        A  1.10  1.20
0        B  1.11  1.22
1        A  1.10  1.20
1        B  1.11  1.22
2        A  1.10  1.20
2        B  1.11  1.22


C:\Users\lukik\AppData\Local\Temp\ipykernel_18400\3319343290.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_reindexed.stack(level = 0)
